In [14]:
import sys
%cd variational_inference_for_longitudinal_data/
sys.path.append('diffusion/stable_diffusion/')
sys.path.append('diffusion/stable_diffusion/model/')
sys.path.append('lib/src/')
import torch
from torch import nn
import os
from matplotlib import pyplot as plt
from sklearn_extra.cluster import KMedoids
import numpy as np


from diffusion.stable_diffusion.latent_diffusion import MyLatentDiffusion, LitLDM
from diffusion.stable_diffusion.model.autoencoder import Autoencoder
from diffusion.stable_diffusion.model.clip_embedder import CLIPTextEmbedder
from diffusion.stable_diffusion.model.unet import UNetModel, _test_time_embeddings

from lib.src.pythae.models import VAE
from lib.src.pythae.models import LVAE_IAF, LVAE_IAF_Config, AutoModel
from lib.src.pythae.models.nn.base_architectures import BaseDecoder
from lib.scripts.utils import Encoder_Chairs,Decoder_Chairs, My_Dataset, My_MaskedDataset, make_batched_masks



from geometric_perspective_on_vaes.sampling import build_metrics, log_pi

def load_config_unet(config):
    return UNetModel(
        in_channels=config['in_channels'],
        out_channels=config['out_channels'],
        channels=config['channels'],
        n_res_blocks=config['n_res_blocks'],
        attention_levels=config['attention_levels'],
        channel_multipliers=config['channel_multipliers'],
        n_heads=config['n_heads'],
    )

%reload_ext autoreload
%autoreload 2

!nvidia-smi

[Errno 2] No such file or directory: 'variational_inference_for_longitudinal_data/'
/users/eleves-a/2020/meilame.tayebjee/variational_inference_for_longitudinal_data
Fri May 31 14:39:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.78                 Driver Version: 550.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A2000 12GB          Off |   00000000:01:00.0 Off |                  Off |
| 30%   35C    P8             11W /   70W |    1142MiB /  12282MiB

In [2]:
PATH_DATA = 'lib/my_data/sprites/Sprites_train.pt'


train_data = torch.load(os.path.join(PATH_DATA))[:-1000]
eval_data = torch.load(os.path.join(PATH_DATA), map_location="cpu")[-1000:]
#test_data = torch.load(os.path.join('lib/my_data/sprites/Sprites_test.pt'), map_location="cpu")

print(train_data.shape)
train_data = train_data.permute(0, 1, 4, 2, 3)
eval_data = eval_data.permute(0, 1, 4, 2, 3)
#test_data = test_data.permute(0, 1, 4, 2, 3)
print(train_data.shape)

train_seq_mask = torch.ones(train_data.shape[:2], requires_grad=False).type(torch.bool)
eval_seq_mask = torch.ones(eval_data.shape[:2], requires_grad=False).type(torch.bool)
#test_seq_mask = torch.ones(test_data.shape[:2], requires_grad=False).type(torch.bool)
train_pix_mask = torch.ones_like(train_data, requires_grad=False).type(torch.bool)
eval_pix_mask = torch.ones_like(eval_data, requires_grad=False).type(torch.bool)
#test_pix_mask = torch.ones_like(test_data, requires_grad=False).type(torch.bool)

train_dataset = My_MaskedDataset(train_data, train_seq_mask, train_pix_mask)
eval_dataset = My_MaskedDataset(eval_data, eval_seq_mask, eval_pix_mask)
#test_dataset = My_MaskedDataset(test_data, test_seq_mask, test_pix_mask)


NUM_WORKERS = 12
train_loader = torch.utils.data.DataLoader(train_data, batch_size=200, shuffle=True)
val_loader = torch.utils.data.DataLoader(eval_data, batch_size=200, shuffle=True)

torch.Size([8000, 8, 64, 64, 3])
torch.Size([8000, 8, 3, 64, 64])


In [3]:
# encoder = Encoder_Chairs(config)
# decoder = Decoder_Chairs(config)
# vae = LVAE_IAF(config, encoder, decoder)
PATH_VAE_FOLDER = 'pre-trained_vae/VAE_training_2024-05-30_17-33-43-latdim12/final_model'
PATH_DIFFUSION_CKPT = 'ldm/lightning_logs/version_30/checkpoints/epoch=44-step=900.ckpt'




device = 'cuda'
vae = VAE.load_from_folder(PATH_VAE_FOLDER).to(device)
vae.eval()

# in_channels = 3
# out_channels = 3
# channels = 32
# n_res_blocks = 2
# attention_levels = [2]
# channel_multipliers = (1, 2, 4)
# n_heads = 16


in_channels = 3
out_channels = 3
channels = 64
n_res_blocks = 4
attention_levels = [0]
channel_multipliers = [1]
n_heads = 4

unet_config = {
    'in_channels': in_channels,
    'out_channels': out_channels,
    'channels': channels,
    'n_res_blocks': n_res_blocks,
    'attention_levels': attention_levels,
    'channel_multipliers': channel_multipliers,
    'n_heads': n_heads,
}

unet = load_config_unet(unet_config)

latent_scaling_factor = 1
n_steps = 1000
linear_start =  0.00085
linear_end = 0.012

input_dim = (3, 64, 64)
f = 32 #subsampling factor
latent_dim = 3* (64 // f) * (64 // f)
print('Latent dim:', latent_dim)


latent_diffusion = MyLatentDiffusion(unet, None, latent_scaling_factor, n_steps, linear_start, linear_end)
print('Number of parameters in the diffusion model: ', sum(p.numel() for p in latent_diffusion.parameters() if p.requires_grad))

model = LitLDM.load_from_checkpoint(PATH_DIFFUSION_CKPT, ldm = latent_diffusion, vae = vae, latent_dim = latent_dim, lr = 6e-4).to('cuda')
diffusion = model.ldm



Latent dim: 12
Number of parameters in the diffusion model:  2223043


In [10]:
def retrieveG(vae, train_data, verbose = False, device = 'cpu'):
    last_obs_train = train_data[:, -1, :, :, :].to(device)
    loader = torch.utils.data.DataLoader(last_obs_train, batch_size=200, shuffle=False)
    mu = []
    log_var = []
    vae = vae.to(device)
    with torch.no_grad():
        for _ , x in enumerate(loader):

            data = x.data

            out = vae.encoder(data.to(device))

            mu_data, log_var_data = out.embedding, out.log_covariance

            mu.append(mu_data)
            log_var.append(log_var_data)

    mu = torch.cat(mu)
    log_var = torch.cat(log_var)

    if verbose:
        print('Running Kmedoids')
        print(mu.shape)

    kmedoids = KMedoids(n_clusters=100).fit(mu.detach().cpu().numpy())
    medoids = torch.tensor(kmedoids.cluster_centers_).to(device)
    centroids_idx = kmedoids.medoid_indices_ #

    if verbose:
        print("Finding temperature")
        
    eps_lf = 0.01
    lbd = 0.01
    T = 0
    T_is = []
    for i in range(len(medoids)-1):
        mask = torch.tensor([k for k in range(len(medoids)) if k != i])
        dist = torch.norm(medoids[i].unsqueeze(0) - medoids[mask], dim=-1)
        T_i =torch.min(dist, dim=0)[0]
        T_is.append(T_i.item())

    T = np.max(T_is)

    if verbose: 
        print('Best temperature found: ', T)
        print('Building metric')

    vae = build_metrics(vae, mu, log_var, centroids_idx, T=T, lbd=lbd)

    return vae.G_sampl, log_var

In [5]:
def log_p_j_hat(j, z, zT_samples, alpha_bars, tau = 125):
    """

    """

    
    alpha_bar_j = alpha_bars[j*tau]
    log_density = torch.sum( (z - zT_samples)**2 / (2 * (1 - alpha_bar_j)**2), dim=-1)

    return log_density.mean()

In [8]:
zT_samples = torch.load('zT_samples.pt').to('cpu')
z = torch.randn(1, latent_dim)
alpha_bars = 0.5*torch.ones(1000)
log_p_j_hat(1, z, zT_samples, alpha_bars)


tensor(40.8740)

In [11]:
G, log_var= retrieveG(vae, train_data, verbose = True)

Running Kmedoids
torch.Size([8000, 12])
Finding temperature
Best temperature found:  1.8156334161758423
Building metric


In [12]:
(log_var > 3*1e-2).sum()/ 8000

tensor(0.4570)

In [13]:
z = vae.encoder(train_data[0, -1, :, :, :].unsqueeze(0).to('cpu')).embedding
log_pi(vae, z)

tensor([-27.6306], device='cuda:0')

## TO DO

#ergonomiser le Riemaniann sampling
- inspecter LVIAF
- train !

Possible pain points : le diffusion backward très bof (at least dans les intemédiaires). Après c'est du sampling... wait and see

30/05
Le problème du vanishing G a été résolu en passant à une dim latente de 12.
Il faut améliorer le diffusion model !
Eventuellemnt passer à du 16, puis 1, 4, 4 pour avoir des étages ce qui  n'est pas le cas now. Checker si on a pas de vansihing G.